In [1]:
from pdb import set_trace as debugger
from tqdm import tqdm
import numpy as np
import pandas
    
def upload_cifar(filename):
    import cPickle
    fo = open(filename, 'rb')
    data = cPickle.load(fo)
    fo.close()

    features = data['data']
    labels = data['labels']
    labels = np.atleast_2d( labels ).T 
    
    # squash classes 0-4 into class 0, and squash classes 5-9 into class 1
    labels[ labels < 5 ] = 0 
    labels[ labels >= 5 ] = 1 
    return features, labels

def upload_iris(filename):
    data = pandas.read_csv(filename)
    m = data.as_matrix()
    labels = m[:,0]
    labels[ labels==2 ] = 1  # squash class 2 into class 1
    labels = np.atleast_2d( labels ).T 
    features = m[:,1:5]
    return features, labels

class Perceptron:
    
    def __init__(self, features, labels, c = 1, theta = 0):
        self.c = c
        self.theta = theta
        # Add a bias input into every instance
        self.features = np.insert(features, features.shape[1], 1, axis=1)
        self.labels = labels  
        # Create a (1,n+1) matrix of weights, +1 for bias input
        self._w = np.matrix(np.random.rand(features.shape[1] + 1))
        
    def run(self):
        for _ in tqdm(xrange(100)):
            for instance, label in zip(self.features, self.labels):
                # Get our prediction 
                z_pred = int(self._w.dot(instance).item() > self.theta)
                # Compare to label and adjust weights if necessary
                if label.item() != z_pred:
                    self.adjust_weights(label.item() - z_pred, instance)

    def adjust_weights(self, learning_direction, instance):
        weight_changes = np.multiply(instance, (self.c * learning_direction))
        self._w = np.add(self._w, weight_changes)
        
        
    def plot(self):
        pass

features, labels = upload_iris("Fisher.csv")
iris_perceptron = Perceptron(features, labels)
iris_perceptron.run()

#features, labels = upload_cifar('cifar-10-batches-py/data_batch_1')
#cifar_perceptron = Perceptron(features, labels, c=0.001)
#cifar_perceptron.run()


100%|██████████| 100/100 [00:00<00:00, 338.20it/s]
